# Assignment 1 (MMP II)

### Name: Syed Haider Ali
### Dept: DPAM
### Reg No: 07-3-1-041-2021
### Batch: 2021-2025

# Heat Equation Solution in One Dimension



## Background

### The Heat Equation: Understanding Heat Flow
The heat equation is a fundamental equation in physics that governs how heat diffuses or flows within a material over time. It describes the relationship between the temperature distribution u(x, t) at a point x and time t with the material's properties and heat sources.

**Significance**:

The heat equation plays a crucial role in various scientific and engineering applications. Here's why it's important:

-> **Predicting temperature distribution**: It allows us to predict how temperatures will change within a system over time, which is essential for designing and optimizing heat transfer processes. Examples include designing heat exchangers, analyzing insulation effectiveness, and understanding thermal behavior in buildings.

-> **Understanding thermal response**: By solving the heat equation, we can understand how a material responds to external thermal loads. This helps predict the risk of overheating, optimize cooling systems, and analyze the thermal behavior of machinery.

-> **Modeling various phenomena**: The heat equation forms the basis for modeling other heat transfer processes like convection and radiation when coupled with appropriate boundary conditions.



## Importing Necessary  Libraries

In [16]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline




### Initial Conditions and Parameters

In [17]:
# Define initial condition
f1 = lambda x: 1
f2 = lambda x: np.exp(-x)  # Exponential function (can be changed)
f3 = lambda x: np.cos(x)
f4 = lambda x: np.exp(-x**2)

# Set parameters
L = np.pi  # Length of the rod
a = 0.1  # Thermal diffusivity
t_max = 1  # Maximum time

# Generate spatial and temporal points
X = np.linspace(0, L, 100)
T = np.linspace(0, t_max, 50)
t_snapshots = [0.25, 0.5, 0.75]  # Time points for snapshots


In [18]:
# Create the simulation environment
env = simpy.Environment()

F = [f1,f2,f3,f4]


## Generating 2D plots with time instants

### Comparing Behaviour of different f(x) interactively

In [19]:
def heat_equation(L, a, f, X, T):
  """
  Simulates the solution of the heat equation using separation of variables.

  Args:
      L: Length of the rod.
      a: Thermal diffusivity.
      f: Initial temperature function.
      x: Spatial points.
      t: Time point.

  Returns:
      The temperature at position x and time t.
  """
  n_terms = 100  # Number of terms in the infinite series
  eigenvalues = np.pi**2 * np.arange(1, n_terms + 1) / (L**2)
  coefficients = np.zeros(n_terms)
  for n in range(n_terms):
    coefficients[n] = 2 / L * np.trapz(f(X) * np.sin(n * np.pi * X / L), X)
  u = np.zeros_like(X)
  for n in range(n_terms):
    u += coefficients[n] * np.exp(-eigenvalues[n] * a * T) * np.sin(n * np.pi * X / L)
  return u


### Interactive Plotting

Here we create an interactive plot by inserting a time slider to really observe the evolution of the system with time for different f(x)

**Note: If the below cell is not displaying run the notebook again** 

In [20]:

def update_plot(T=1, a=0.1):
    u1 = heat_equation(L, a, f1, X, T)
    u2 = heat_equation(L, a, f2, X, T)
    u3 = heat_equation(L, a, f3, X, T)
    u4 = heat_equation(L, a, f4, X, T)

    plt.clf()  # Clear the previous plot
    plt.plot(X, u1, label=r"$f(x) = 1$")
    plt.plot(X, u2, label=r"$f(x) = e^{-x}$")
    plt.plot(X, u3, label=r"$f(x) = \cos(x)$")
    plt.plot(X, u4, label=r"$f(x) = e^{-x^2}$")
    plt.xlabel("x")
    plt.ylabel("U(x,t)")
    plt.title("Heat Equation Solution (t = %.2f)" % T)
    plt.legend()
    plt.show()

# Create a slider widget for time
time_slider = FloatSlider(min=0, max=10, step=0.01, value=0.5, description='Time')

# Create a slider widget for Thermal diffusivity
a_slider = FloatSlider(min=0, max=1, step=0.001, value=0.1, description='Thermal diffusivity')

# Connect the slider to the update function
interact(update_plot, T=time_slider, a=a_slider)

interactive(children=(FloatSlider(value=0.5, description='Time', max=10.0, step=0.01), FloatSlider(value=0.1, …

<function __main__.update_plot(T=1, a=0.1)>

## Generating 3D plots of the Heat Equation

In [21]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider

# Define symbols and variables
x, t = sp.symbols('x t')
n = sp.symbols('n', integer=True)  # Define n as a symbol

def plot_3d(Func, L, LABEL):

    def TD_update_plot(a=0.1):
        # Define the thermal diffusivity parameter
        a_value = a
    
        # Define the solution expression
        u_n = (2/L) * sp.integrate(Func * sp.sin((n * np.pi * x)/L), (x, 0, L)) * sp.sin((n * np.pi * x)/L) * sp.exp(-((n*np.pi*a_value)/L)**2 * t)

        solution = sp.Sum(u_n, (n, 1, 100))

        # Compute the solution
        solution = solution.doit()

        # Create a lambdified function for numerical evaluation
        u_func = sp.lambdify((x, t), solution, 'numpy')

        # Define the range for x and t
        x_values = np.linspace(0, np.pi, 100)
        t_values = np.linspace(0, 1, 100)

        # Create meshgrid for 3D plotting
        X, T = np.meshgrid(x_values, t_values)

        # Compute the solution for the meshgrid
        Z = np.zeros_like(X)
        for i in range(1, 11):  # Summation from n=1 to n=10
            Z += u_func(X, T)

        # Plot
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(X, T, Z, cmap='viridis')
        ax.set_xlabel('x')
        ax.set_ylabel('t')
        ax.set_zlabel('u(x,t)')
        ax.set_title('Solution of 1D Heat Equation '+LABEL+ 'with a = {:.3f}'.format(a_value))
        plt.show()

    # Create a slider widget for Thermal diffusivity
    a_slider = FloatSlider(min=0, max=1, step=0.001, value=0.1, description='Thermal diffusivity')

    # Connect the slider to the update function
    interact(TD_update_plot, a=a_slider)


In [22]:
plot_3d(sp.sin(x), L=0.5*np.pi, LABEL=r"$f(x) = \sin(x)$ ")
plot_3d(sp.cos(x), L=0.5*np.pi, LABEL=r"$f(x) = \cos(x)$ ")
plot_3d(sp.exp(-x), L=np.pi, LABEL=r"$f(x) = e^{-x}$ ")
plot_3d(1, L=np.pi, LABEL=r"$f(x) =1$ ")

interactive(children=(FloatSlider(value=0.1, description='Thermal diffusivity', max=1.0, step=0.001), Output()…

interactive(children=(FloatSlider(value=0.1, description='Thermal diffusivity', max=1.0, step=0.001), Output()…

interactive(children=(FloatSlider(value=0.1, description='Thermal diffusivity', max=1.0, step=0.001), Output()…

interactive(children=(FloatSlider(value=0.1, description='Thermal diffusivity', max=1.0, step=0.001), Output()…

## Conclusion

In this notebook, we explored the solution of the one-dimensional heat equation using separation of variables and numerical methods. We investigated the behavior of temperature profiles along a rod at a fixed time and thermal diffusivity, considering various initial conditions.

For the initial condition **f(x) = exp(x)**, the temperature profile exhibited a rapidly increasing trend along the rod, followed by a sharp decline towards zero temperature at the end of the rod. This behavior reflects the initial rapid temperature rise and subsequent diffusion-driven decrease in temperature gradient.

For the initial condition **f(x) = exp(-x)**  the temperature profile along the rod exhibits a rapidly decreasing trend from the left end (where x = 0) towards the right end (where x = π).This behavior reflects the initial sharp decline in temperature as we move along the rod, starting from a higher temperature at the left end and decreasing exponentially towards zero temperature at the right end.


With the initial condition **f(x) = 1**, the temperature profile remained nearly constant along the rod, indicating minimal change in temperature over distance. This result is consistent with the concept of a uniform initial temperature distribution maintaining its constancy over time.

For the initial condition **f(x) = cos(x)**, the temperature profile oscillated between positive and negative values, gradually decreasing in magnitude over time. The oscillatory behavior smoothed out as heat diffused along the rod, eventually leading to a temperature distribution that approached zero.

Additionally, we observed a similar behavior for the initial condition **f(x) = exp(-x^2), where the temperature profile exhibited oscillations resembling those of the cosine function but with weaker amplitude.

These results highlight the influence of initial conditions on the evolution of temperature distributions in materials subjected to heat conduction. By studying the behavior of temperature profiles under different initial conditions, we gain valuable insights into the thermal dynamics of one-dimensional systems.